___
# Projeto 2 - Ciência dos Dados
Grupo: Bruno Besnosik, Eduardo Zetone, Enzo Del Manto, Guilherme Pinheiro

## Objetivo do projeto:

- Testar a possibilidade de prever o desempenho de um aluno, baseando-se em suas relações sociais e acadêmicas.
- O nosso target é quantitativo (G3), já que queremos prever uma informação numérica. 

In [2]:
# Importar bibliotecas necessárias
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
from math import *

In [5]:
# Abrir o dataset e converter em dataframe
df = pd.read_csv('data/student-mat.csv')

## Informações do dataset
- O dataset foi gerado a partir de um questionário feito com estudantes da disciplina matemática do ensino médio.   
- O arquivo contém informações pessoais dos estudantes, tanto sobre sua vida social e acadêmica, quanto sobre suas relações interfamiliares, amorosas e etc.

https://www.kaggle.com/uciml/student-alcohol-consumption

## Índice do dataset
- school - Escola do estudante (binário: 'GP' - Gabriel Pereira ou 'MS' - Mousinho da Silveira)
- sex - Gênero (binário: 'F' - feminino ou 'M' - masculino)
- age - Idade (numérico: de 15 até 22)
- address - Tipo de endereço (binário: 'U' - urbano ou 'R' - rural)
- famsize - Número de membros na família (binário: 'LE3' (0) - menor ou igual a 3 ou 'GT3' (1) - maior que 3)
- Pstatus - Estado de coabitação dos pais (binário: 'T' - Morando juntos ou 'A' - Morando separados)
- Medu - Nível de educação da mãe (numérico: 0 - Nenhuma, 1 - Ensino Primário (Quarto ano), 2 – Quinto ao nono ano, 3 – Ensino secundário ou 4 – Ensino Superior)
- Fedu - Nível de educação do pai (numérico: 0 - Nenhuma, 1 - Ensino Primário (Quarto ano), 2 – Quinto ao nono ano, 3 – Ensino secundário ou 4 – Ensino Superior)
- Mjob - Trabalho da mãe (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
- Fjob - Trabalho do pai (nominal: 'teacher', 'health' care related, civil 'services' (e.g. administrative or police), 'at_home' or 'other')
- reason - Razão de ter escolhido essa escola (nominal: close to 'home', school 'reputation', 'course' preference or 'other')
- guardian - Guardião Legal do estudante (nominal: 'mother', 'father' or 'other')
- traveltime -Tempo de viagem de casa até a escola (numérico: 1 - Até 15 minutos, 2 - 15 a 30 minutos, 3 - 30 minutos até 1 hora, or 4 - mais que 1 hora)
- studytime - Tempo de estudo semanal (numérico: 1 - Até 2 horas, 2 - 2 a 5 horas, 3 - 5 a 10 horas, or 4 - mais que 10 horas)
- failures - Número de reprovamentos anteriores (numérico: n se 1<=n<3, caso contrário-4)
- schoolsup - Suporte educacional extra (binário: yes or no)
- famsup - Suporte educacional da família (binário: yes or no)
- paid - Se faz aulas particulares da disciplina (binário: yes or no)
- activities - Se faz atividades extra-curriculares (binário: yes or no)
- nursery - Se fez escola de enfermagem (binário: yes or no)
- higher - Se quer ter ensino superior (binário: yes or no)
- internet - Se tem acesso à internet em casa (binário: yes or no)
- romantic - Se está em um relacionamento amoroso (binário: yes or no)
- famrel - Qualidade das relações familiares (numérico: De 1 - muito ruins até 5 - excelentes)
- freetime - Tempo livre depois da escola (numérico: De 1 - pouco até 5 - muito)
- goout - Sair com os amigos (numérico: De 1 - pouco até 5 - muito)
- Dalc - Consumo de álcool em dia de semana (numérico: De 1 - baixo até 5 - elevado)
- Walc - Consumo de álcool no fim de semana (numérico: from 1 - baixo até 5 - elevado)
- health - Estado de saúde atual (numérico: De 1 - Muito ruim até 5 - Muito bom)
- absences - Número de faltas na escola (numérico: De 0 até 93)
- G1: Nota no primeiro semestre (númerico: De 0 até 20) 
- G2: Nota no segundo semestre (númerico: De 0 até 20)
- G3: Nota final (numérico: De 0 até 20, output target) 